In [117]:
import sympy
import numpy as np
from tqdm import tqdm
import dill

dill.settings["recurse"] = True
from sympy import Rational as R

In [118]:
(
    reference_x,
    reference_y,
    reference_x_1,
    reference_y_1,
    reference_x_2,
    reference_y_2,
    reference_x_3,
    reference_y_3,
) = sympy.symbols("x y x_1 y_1 x_2 y_2 x_3 y_3")

rotation_matrix = sympy.Matrix([[+R(0), +R(1)], [-R(1), +R(0)]])

t1_hat = sympy.Matrix([-R(1), +R(1)]) / sympy.sqrt(2)
t2_hat = sympy.Matrix([+R(0), +R(1)])
t3_hat = sympy.Matrix([+R(1), +R(0)])

n1_hat = rotation_matrix @ t1_hat
n2_hat = rotation_matrix @ t2_hat
n3_hat = rotation_matrix @ t3_hat

### Get basis functions

In [119]:
monomial_basis = sympy.Matrix(
    [
        1,
        reference_x,
        reference_y,
        reference_x**2,
        reference_x * reference_y,
        reference_y**2,
    ]
)

In [120]:
Vander = sympy.Matrix(np.zeros((6, 6)))

for i, basis in enumerate(monomial_basis):

    v1 = {reference_x: 0, reference_y: 0}
    Vander[i, 0] = basis.subs(v1)

    v2 = {reference_x: 1, reference_y: 0}
    Vander[i, 1] = basis.subs(v2)

    v3 = {reference_x: 0, reference_y: 1}
    Vander[i, 2] = basis.subs(v3)

    e1 = {reference_x: R(1, 2), reference_y: R(1, 2)}
    dn1 = n1_hat[0] * basis.diff(reference_x) + n1_hat[1] * basis.diff(reference_y)
    Vander[i, 3] = dn1.subs(e1)

    e2 = {reference_x: 0, reference_y: R(1, 2)}
    dn2 = n2_hat[0] * basis.diff(reference_x) + n2_hat[1] * basis.diff(reference_y)
    Vander[i, 4] = dn2.subs(e2)

    e3 = {reference_x: R(1, 2), reference_y: 0}
    dn3 = n3_hat[0] * basis.diff(reference_x) + n3_hat[1] * basis.diff(reference_y)
    Vander[i, 5] = dn3.subs(e3)

In [127]:
morley_basis = Vander.inv() @ monomial_basis

### Get mapping functions

In [143]:
monomial_basis = sympy.Matrix(
    [
        1,
        reference_x,
        reference_y,
    ]
)

V = sympy.zeros(3, 3)

for dofidx, basis in enumerate(monomial_basis):

    V[dofidx, 0] = basis.subs({reference_x: 0, reference_y: 0})
    V[dofidx, 1] = basis.subs({reference_x: 1, reference_y: 0})
    V[dofidx, 2] = basis.subs({reference_x: 0, reference_y: 1})

mapping_basis = V.inv() @ monomial_basis

global_x = (
      mapping_basis[0] * reference_x_1
    + mapping_basis[1] * reference_x_2
    + mapping_basis[2] * reference_x_3
)
global_y = (
      mapping_basis[0] * reference_y_1
    + mapping_basis[1] * reference_y_2
    + mapping_basis[2] * reference_y_3
)

mapping_function = sympy.Matrix([global_x, global_y])

In [144]:
J = mapping_function.jacobian([reference_x, reference_y]).inv()

In [145]:
J_cofactor_T = mapping_function.jacobian([reference_x, reference_y]).cofactor_matrix().T

# Necessary symbols

In [146]:
G1_hat = n1_hat.row_join(t1_hat).T
G2_hat = n2_hat.row_join(t2_hat).T
G3_hat = n3_hat.row_join(t3_hat).T

l1 = ((reference_x_3 - reference_x_2) ** 2 + (reference_y_3 - reference_y_2) ** 2) ** R(
    1, 2
)
l2 = ((reference_x_3 - reference_x_1) ** 2 + (reference_y_3 - reference_y_1) ** 2) ** R(
    1, 2
)
l3 = ((reference_x_2 - reference_x_1) ** 2 + (reference_y_2 - reference_y_1) ** 2) ** R(
    1, 2
)

t1 = sympy.Matrix([reference_x_3 - reference_x_2, reference_y_3 - reference_y_2]) / l1
t2 = sympy.Matrix([reference_x_3 - reference_x_1, reference_y_3 - reference_y_1]) / l2
t3 = sympy.Matrix([reference_x_2 - reference_x_1, reference_y_2 - reference_y_1]) / l3

n1 = rotation_matrix @ t1
n2 = rotation_matrix @ t2
n3 = rotation_matrix @ t3

G1 = n1.row_join(t1).T
G2 = n2.row_join(t2).T
G3 = n3.row_join(t3).T

B1 = G1_hat @ J.inv().T @ G1.T
B2 = G2_hat @ J.inv().T @ G2.T
B3 = G3_hat @ J.inv().T @ G3.T

f_interpolation_extended = sympy.Matrix(sympy.symbols("f_1:10"))

f_interpolation = sympy.Matrix(f_interpolation_extended[:6])

f_interpolation[3] = (
    n1[0] * f_interpolation_extended[3] + n1[1] * f_interpolation_extended[4]
)
f_interpolation[4] = (
    n2[0] * f_interpolation_extended[5] + n2[1] * f_interpolation_extended[6]
)
f_interpolation[5] = (
    n3[0] * f_interpolation_extended[7] + n3[1] * f_interpolation_extended[8]
)

### Calculate bilinear form and right values for forward pushed reference element

In [147]:
N = len(morley_basis)

weak_form_functional_xxyy = [[0 for i in range(N)] for j in range(N)]
weak_form_functional_xyxy = [[0 for i in range(N)] for j in range(N)]
weak_form_right_part =      [[0 for i in range(N)] for j in range(N)]

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    
    first = morley_basis[idx]
    second = morley_basis[jdx]

    # -----------------------U------------------------------------------------------------------------------
    u_x = first.diff(reference_x) * J_cofactor_T.row(0)[0] + first.diff(reference_y) * J_cofactor_T.row(1)[0]
    u_y = first.diff(reference_x) * J_cofactor_T.row(0)[1] + first.diff(reference_y) * J_cofactor_T.row(1)[1]

    u_xx = u_x.diff(reference_x) * J_cofactor_T.row(0)[0] + u_x.diff(reference_y) * J_cofactor_T.row(1)[0]
    u_xy = u_x.diff(reference_x) * J_cofactor_T.row(0)[1] + u_x.diff(reference_y) * J_cofactor_T.row(1)[1]

    u_yy = u_y.diff(reference_x) * J_cofactor_T.row(0)[1] + u_y.diff(reference_y) * J_cofactor_T.row(1)[1]
    # ------------------------------------------------------------------------------------------------------

    # -----------------------V------------------------------------------------------------------------------
    v_x = second.diff(reference_x) * J_cofactor_T.row(0)[0] + second.diff(reference_y) * J_cofactor_T.row(1)[0]    
    v_y = second.diff(reference_x) * J_cofactor_T.row(0)[1] + second.diff(reference_y) * J_cofactor_T.row(1)[1]

    v_xx = v_x.diff(reference_x) * J_cofactor_T.row(0)[0] + v_x.diff(reference_y) * J_cofactor_T.row(1)[0]
    v_xy = v_x.diff(reference_x) * J_cofactor_T.row(0)[1] + v_x.diff(reference_y) * J_cofactor_T.row(1)[1]

    v_yy = v_y.diff(reference_x) * J_cofactor_T.row(0)[1] + v_y.diff(reference_y) * J_cofactor_T.row(1)[1]
    # ------------------------------------------------------------------------------------------------------

    # -----------------------bilinear-form------------------------------------------------------------------
    A = u_xx * v_xx
    B = u_xy * v_xy
    b = u_xx * v_yy
    C = u_yy * v_yy
    # ------------------------------------------------------------------------------------------------------

    # -----------------------integrals----------------------------------------------------------------------
    sym_int_B = sympy.integrate(
        A + 2 * B + C, (reference_y, 0, 1 - reference_x), (reference_x, 0, 1)
    )
    sym_int_b = sympy.integrate(
        A + 2 * b + C, (reference_y, 0, 1 - reference_x), (reference_x, 0, 1)
    )
    sym_int_A = sympy.integrate(
        first * second, (reference_y, 0, 1 - reference_x), (reference_x, 0, 1)
    )

    weak_form_functional_xyxy[idx][jdx] = sym_int_B
    weak_form_functional_xxyy[idx][jdx] = sym_int_b
    weak_form_right_part[idx][jdx] = sym_int_A
# -----------------------------------------------------------------------------------------------------

weak_form_functional_xxyy = sympy.Matrix(weak_form_functional_xxyy)
weak_form_functional_xyxy = sympy.Matrix(weak_form_functional_xyxy)
weak_form_right_part = sympy.Matrix(weak_form_right_part)

# The determinant of the Jacobian does not depend on (x, y) so we can take it outside to make the symbolic integration faster
weak_form_functional_xxyy = weak_form_functional_xxyy * abs(J_cofactor_T.det()) / J_cofactor_T.det()**4 # J_inv.det() == J_cofactor_T.det()
weak_form_functional_xyxy = weak_form_functional_xyxy * abs(J_cofactor_T.det()) / J_cofactor_T.det()**4
weak_form_right_part = weak_form_right_part * abs(J_cofactor_T.det())

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [01:20<00:00,  2.24s/it]


# Define transformation matrices

### $V^{c}$

In [148]:
V_c = sympy.diag(
    1,
    1,
    1,
    B1,
    B2,
    B3,
)

### $E$

In [149]:
E = [
    [1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
]
E = sympy.Matrix(E)

### $D$

In [150]:
D = sympy.Matrix(
    [
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, -1 / l1, 1 / l1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [-1 / l2, 0, 1 / l2, 0, 0, 0],
        [0, 0, 0, 0, 0, 1],
        [-1 / l3, 1 / l3, 0, 0, 0, 0],
    ]
)

### $M$

In [151]:
V = E @ V_c @ D
M = V.T

### Apply transformations

In [152]:
weak_form_functional_xxyy = M @ weak_form_functional_xxyy @ M.T

In [153]:
weak_form_functional_xyxy = M @ weak_form_functional_xyxy @ M.T

In [154]:
weak_form_right_part = M @ weak_form_right_part @ M.T @ f_interpolation

In [155]:
lambdify_symbols = [
    reference_x_1,
    reference_y_1,
    reference_x_2,
    reference_y_2,
    reference_x_3,
    reference_y_3,
]

In [156]:
weak_form_functional_xxyy_lambdified = sympy.lambdify(
    lambdify_symbols, weak_form_functional_xxyy, cse=True
)

In [157]:
weak_form_functional_xyxy_lambdified = sympy.lambdify(
    lambdify_symbols, weak_form_functional_xyxy, cse=True
)

In [158]:
weak_form_right_part_lambdified = sympy.lambdify(
    [*lambdify_symbols, *f_interpolation_extended], weak_form_right_part, cse=True
)

In [159]:
interpolation_function = ((M@morley_basis).T @ f_interpolation)[0, 0]

interpolation_function_lambdified = sympy.lambdify(
    [*lambdify_symbols, *f_interpolation_extended, reference_x, reference_y],
    interpolation_function,
    cse=True,
)

In [160]:
f_interpolation_normal = sympy.Matrix(sympy.symbols("f_1:7"))

interpolation_function_normal = ((M@morley_basis).T @ f_interpolation_normal)[0, 0]

interpolation_function_normal_lambdified = sympy.lambdify(
    [*lambdify_symbols, *f_interpolation_normal, reference_x, reference_y],
    interpolation_function_normal,
    cse=True,
)

In [161]:
mapping_function_lambdified = sympy.lambdify(
    [*lambdify_symbols, reference_x, reference_y], mapping_function[:, 0], cse=True
)

### Save the results

In [162]:
dill.dump(
    weak_form_functional_xxyy_lambdified,
    open(
        "../calculations/nonconforming_quadratic_morley_weak_form_functional_xxyy", "wb"
    ),
)
dill.dump(
    weak_form_functional_xyxy_lambdified,
    open(
        "../calculations/nonconforming_quadratic_morley_weak_form_functional_xyxy", "wb"
    ),
)
dill.dump(
    weak_form_right_part_lambdified,
    open("../calculations/nonconforming_quadratic_morley_weak_form_right_part", "wb"),
)

dill.dump(
    interpolation_function_lambdified,
    open("../calculations/nonconforming_quadratic_morley_basis", "wb"),
)

dill.dump(
    interpolation_function_normal_lambdified,
    open("../calculations/nonconforming_quadratic_morley_basis_normal", "wb"),
)


dill.dump(
    mapping_function_lambdified,
    open("../calculations/nonconforming_quadratic_morley_mapping_function", "wb"),
)